In [1]:
import pandas as pd
from prediction_market_agent_tooling.markets.seer.seer import SeerAgentMarket
from prediction_market_agent_tooling.markets.seer.seer_subgraph_handler import (
    SeerSubgraphHandler,
    HexBytes,
    FilterBy,
    SortBy,
)

from prediction_market_agent_tooling.tools.tokens.usd import (
    get_token_in_usd, get_usd_in_token
)
from prediction_market_agent_tooling.markets.omen.omen_constants import SDAI_CONTRACT_ADDRESS


from prediction_market_agent_tooling.gtypes import (
    USD,
    CollateralToken,
    OutcomeStr,
    OutcomeToken,
    Wei,
)

from prediction_market_agent_tooling.tools.cow.cow_order import (
    get_buy_token_amount_else_raise,
)

from enum import Enum
from typing import Dict


import asyncio
import nest_asyncio

import pandas as pd
from IPython.display import HTML, display


nest_asyncio.apply()

2025-07-11 10:50:33.109 | INFO     | prediction_market_agent_tooling.loggers:patch_logger:149 - Patched logger for default format.


In [2]:
subgraph_handler = SeerSubgraphHandler()

In [3]:
categorical_markets_data = subgraph_handler.get_markets(
    filter_by=FilterBy.OPEN,
    sort_by=SortBy.HIGHEST_LIQUIDITY,
    outcome_supply_gt_if_open=Wei(0),
    include_conditional_markets=False,
    include_categorical_markets=True,
    include_only_scalar_markets=False
)

scalar_markets_data = subgraph_handler.get_markets(
    filter_by=FilterBy.OPEN,
    sort_by=SortBy.HIGHEST_LIQUIDITY,
    outcome_supply_gt_if_open=Wei(0),
    include_conditional_markets=False,
    include_categorical_markets=False,
    include_only_scalar_markets=True
)

all_markets = categorical_markets_data + scalar_markets_data
print(f"Found {len(all_markets)} markets")

all_markets_df = pd.DataFrame([
    {
        'id': market.id.hex(),
        'creator': market.creator,
        'title': market.title,
        'outcomes': market.outcomes,
        'wrapped_tokens': market.wrapped_tokens,
        'parent_outcome': market.parent_outcome,
        'parent_market': market.parent_market,
        'collateral_token': market.collateral_token,
        'condition_id': market.condition_id,
        'opening_ts': market.opening_ts,
        'block_timestamp': market.block_timestamp,
        'has_answers': market.has_answers,
        'payout_reported': market.payout_reported,
        'payout_numerators': market.payout_numerators,
        'outcomes_supply': market.outcomes_supply,
        'upper_bound': market.upper_bound,
        'lower_bound': market.lower_bound,
    }
    for market in all_markets
])


Found 32 markets


In [ ]:
all_markets_sorted_by_liquidity = sorted(all_markets, key=lambda x: x.outcomes_supply, reverse=True)
all_markets_sorted_by_liquidity

In [5]:
sdai_price = get_token_in_usd(USD(1), SDAI_CONTRACT_ADDRESS)

def get_buy_token_amount_cow(bet_amount: USD | CollateralToken, outcome_str: OutcomeStr, market: SeerAgentMarket):
        if outcome_str not in  market.outcomes:
            raise ValueError(
                f"Outcome {outcome_str} not found in market outcomes {market.outcomes}"
            )

        outcome_token = market.get_wrapped_token_for_outcome(outcome_str)
        bet_amount_in_tokens = market.get_in_token(bet_amount)
        try:
            result = get_buy_token_amount_else_raise(
                    sell_amount=bet_amount_in_tokens.as_wei,
                    sell_token=market.collateral_token_contract_address_checksummed,
                    buy_token=outcome_token,
                ).as_token
            print(result, outcome_token)
            return result
        except Exception as e:
            print(f'Error getting cow results for {bet_amount} {outcome_str}')
            print(e)
            return None

def get_buy_token_amount_pools(bet_amount: USD | CollateralToken, outcome_str: OutcomeStr, market: SeerAgentMarket):
    if outcome_str not in  market.outcomes:
        raise ValueError(
            f"Outcome {outcome_str} not found in market outcomes {market.outcomes}"
        )

    outcome_token = market.get_wrapped_token_for_outcome(outcome_str)
    bet_amount_in_tokens = market.get_in_token(bet_amount)
    p = market.get_price_manager()
    try:
        prices=p.get_token_price_from_pools(token=outcome_token)
        print(prices.priceOfCollateralInAskingToken*bet_amount_in_tokens, outcome_token)
    except Exception as e:
        print(f'Error getting pools results for {bet_amount} {outcome_str}')
        print(e)
        return None
    
    return (
                prices.priceOfCollateralInAskingToken * bet_amount_in_tokens
                if prices
                else None
            )



In [6]:
class BetAmount(Enum):
    SMALL = "SMALL"  # 0.1 SDAI
    MEDIUM = "MEDIUM"  # 1 SDAI
    LARGE = "LARGE"  # 10 SDAI
    
    def get_amount(self, base_amount: float) -> float:
        if self == BetAmount.SMALL:
            return base_amount / 10
        elif self == BetAmount.MEDIUM:
            return base_amount
        elif self == BetAmount.LARGE:
            return base_amount * 10
        raise ValueError(f"Unknown bet amount type: {self}")

In [ ]:
market_id_bytes = ##HexBytes('MARKET_HEX')
market = next(filter(lambda x: x.id == market_id_bytes, all_markets_sorted_by_liquidity))
market = SeerAgentMarket.from_data_model_with_subgraph(
    market, SeerSubgraphHandler(), must_have_prices=False
)
for outcome in market.outcomes:
        if 'INVALID' not in outcome.upper():
            cow_results, pools_results = {},{}
            for bet_type in BetAmount:
                bet_amount = bet_type.get_amount(sdai_price)
                cow_results[f'{bet_type}_sdai_price'] = get_buy_token_amount_cow(bet_amount, OutcomeStr(outcome), market)


            
            for bet_type in BetAmount:
                bet_amount = bet_type.get_amount(sdai_price)
                pools_results[f'{bet_type}_sdai_price'] = get_buy_token_amount_pools(bet_amount, OutcomeStr(outcome), market)

            print(f'COW Results: {cow_results}')
            print('--------------------------------')
            print(f'Pools Results: {pools_results}')
            print('--------------------------------')

2025-07-11 10:56:33.116 | WARNING  | prediction_market_agent_tooling.markets.seer.price_manager:get_amount_of_token_in_collateral:101 - Could not get quote for token='0xccf5B9367BEa9159FD75aC329572f9dce58d020f' from Cow, exception e=RetryError(<Future at 0x7478bc707d90 state=finished raised NoLiquidityAvailableOnCowException>). Falling back to pools. 
2025-07-11 10:56:33.338 | WARNING  | prediction_market_agent_tooling.markets.seer.price_manager:get_token_price_from_pools:121 - Could not find a pool for token='0xccf5B9367BEa9159FD75aC329572f9dce58d020f'


0.12265761484097354 0x27827651fb99f4d241F450e93135cAd9Fb7831F4
1.2178703684058978 0x27827651fb99f4d241F450e93135cAd9Fb7831F4
Error getting cow results for 11.878178220193089 Yes
RetryError[<Future at 0x7478bc60f8d0 state=finished raised NoLiquidityAvailableOnCowException>]
0.12276960245522793 0x27827651fb99f4d241F450e93135cAd9Fb7831F4
1.2276960245522794 0x27827651fb99f4d241F450e93135cAd9Fb7831F4
12.276960245522794 0x27827651fb99f4d241F450e93135cAd9Fb7831F4
COW Results: {'BetAmount.SMALL_sdai_price': CollateralToken(0.12265761484097354), 'BetAmount.MEDIUM_sdai_price': CollateralToken(1.2178703684058978), 'BetAmount.LARGE_sdai_price': None}
--------------------------------
Pools Results: {'BetAmount.SMALL_sdai_price': CollateralToken(0.12276960245522793), 'BetAmount.MEDIUM_sdai_price': CollateralToken(1.2276960245522794), 'BetAmount.LARGE_sdai_price': CollateralToken(12.276960245522794)}
--------------------------------
0.6153521904031596 0xbaa447Ef3fAeABEfFDB1Be2d80573459f42d45B8
6.0570

In [ ]:
results = []
for market in all_markets_sorted_by_liquidity[0:100]: #Take 100 markets
    try:
        market = SeerAgentMarket.from_data_model_with_subgraph(
            market, SeerSubgraphHandler(), must_have_prices=False
        )
        print(market.question, market.url, market.outcomes)
    except Exception as e:
        print(f'Error getting market {market.id}')
        print(e)
        continue

    for outcome in market.outcomes:
        if 'INVALID' not in outcome.upper():
            cow_results, pools_results = {},{}
            for bet_type in BetAmount:
                bet_amount = bet_type.get_amount(sdai_price)
                cow_results[f'{bet_type}_sdai_price'] = get_buy_token_amount_cow(bet_amount, OutcomeStr(outcome), market)

    
            
            for bet_type in BetAmount:
                bet_amount = bet_type.get_amount(sdai_price)
                pools_results[f'{bet_type}_sdai_price'] = get_buy_token_amount_pools(bet_amount, OutcomeStr(outcome), market)



            results.append( 
                {
                'outcome_token': OutcomeStr(outcome),
                'outcome_token_wraped': market.get_wrapped_token_for_outcome(OutcomeStr(outcome)),
                'cow': cow_results,
                'pools': pools_results,
                'market': market,
                }
            ) 

print(results)

In [10]:
market_name = set()
for market in all_markets:
    market_name.add(market.title)

print(len(market_name))

market_name = set()
for market in market_name:
    print(market)

print('--------------------------------')
market_name = set()
for result in results:
    market_name.add(result['market'].question)

print(len(market_name))
for market in market_name:
    print(market)



28
--------------------------------
12
How many North Atlantic hurricanes in 2025? [Hurricanes]
Closing Tesla price USD on July 25th 2025 on the NY stock exchange? [noidea]
How many seats will [party name] win in Japan's 2025 House of Council elections? [Seats]
How many people will attend Devcon 8? [individuals]
Will the Argentine government increase (or remove entirely) the monthly limit on individual dollar purchases to at least USD 100,000 by December 31, 2025?
Was Zelenskyy seen wearing a suit between May 22 2025 and June 30 2025?
 What will be the “Overall score” listed for Syria (or its main successor state) in The Economist Democracy Index for 2025? [Index, 2 decimals]
Who will win Bolivian presidential election on 17 August 2025?
Will Kleros Court V2 fully release and officially launch in 2025?
Will Jesus Christ return in 2025?
How many people will be deported (removals, returns & expulsions) from the United States during 2025? [Individuals]
Will Elon Musk be CEO of Tesla on 1 

In [11]:
sizes   = [("Small", "SMALL"), ("Medium", "MEDIUM"), ("Large", "LARGE")]
markets = {} 

for r in results:
    m        = r["market"]
    m_id     = m.id
    outcome  = r["outcome_token"]
    wrapped  = r["outcome_token_wraped"]

    if m_id not in markets:
        markets[m_id] = {
            "Market Question": m.question,
            "Market URL"     : m.url,
            "Market ID"      : m_id,
            "COW Lines"      : [],
            "Pool Lines"     : [],
            "Diff Lines"     : [],
        }

    cow_line, pool_line, diff_line = [f"{outcome} [{wrapped}]: "]*3

    for human, key in sizes:
        cow_raw  = r["cow"  ].get(f"BetAmount.{key}_sdai_price")
        pool_raw = r["pools"].get(f"BetAmount.{key}_sdai_price")

        cow  = float(cow_raw)  if cow_raw  is not None else None
        pool = float(pool_raw) if pool_raw is not None else None
        diff = cow - pool if (cow is not None and pool is not None) else None

        label_html = f"<b>{human}</b>"

        if pool is None:
            pool_part = f"{label_html} null"
        else:
            pool_part_val = f"{pool:.8f}"

        if cow is None:
            cow_part  = f"{label_html} null"
            diff_part = f"{label_html} null"
            pool_part = pool_part if pool is None else f"{label_html} {pool_part_val}"
        else:
            colour_cow  = "green" if diff > 0 else "red"
            colour_pool = "red"   if diff > 0 else "green"
            cow_part  = (
                f'{label_html} '
                f'<span style="color:{colour_cow}; font-weight:bold;">{cow:.8f}</span>'
            )
            diff_part = (
                f'{label_html} '
                f'<span style="color:{colour_cow}; font-weight:bold;">{diff:+.8f}</span>'
            )
            pool_part = (
                f'{label_html} '
                f'<span style="color:{colour_pool}; font-weight:bold;">{pool:.8f}</span>'
            )

        cow_line  += cow_part  + "; "
        pool_line += pool_part + "; "
        diff_line += diff_part + "; "

    markets[m_id]["COW Lines"] .append(cow_line.rstrip("; "))
    markets[m_id]["Pool Lines"].append(pool_line.rstrip("; "))
    markets[m_id]["Diff Lines"].append(diff_line.rstrip("; "))

rows_csv, rows_html = [], []

for m in markets.values():
    rows_csv.append({
        "Market Question": m["Market Question"],
        "Market URL"     : m["Market URL"],
        "COW Values"     : " || ".join(m["COW Lines"]).replace("<span","").replace("</span>","").replace("<b>","").replace("</b>",""),
        "Pool Values"    : " || ".join(m["Pool Lines"]).replace("<span","").replace("</span>","").replace("<b>","").replace("</b>",""),
        "Diff Values"    : " || ".join(m["Diff Lines"]).replace("<span","").replace("</span>","").replace("<b>","").replace("</b>",""),
        "Market ID"      : m["Market ID"],
    })

    rows_html.append({
        "Market Question": m["Market Question"],
        "Market URL"     : f'<a href="{m["Market URL"]}" target="_blank">{m["Market URL"]}</a>',
        "COW Values"     : "<br>".join(m["COW Lines"]),
        "Pool Values"    : "<br>".join(m["Pool Lines"]),
        "Diff Values"    : "<br>".join(m["Diff Lines"]),
        "Market ID"      : m["Market ID"],
    })

df_csv  = pd.DataFrame(rows_csv)
df_disp = pd.DataFrame(rows_html)

csv_name = "market_analysis.csv"
df_csv.to_csv(csv_name, index=False)
print(f"CSV saved → {csv_name}")

display(HTML(df_disp.to_html(escape=False)))

CSV saved → market_analysis.csv


,Market Question,Market URL,COW Values,Pool Values,Diff Values,Market ID
0,"How many people will be deported (removals, returns & expulsions) from the United States during 2025? [Individuals]",https://app.seer.pm/markets/100/0x4446a16765a38a2d1a712d0d4ebf27b3efbdd50a,DOWN [0xa95BfD1e35D1Ce9906A92108A96ccEe1101aBfaa]: Small 0.24872849; Medium 2.48308205; Large 24.41821572UP [0x9C935D4d196037Fc3C37f4BB0a92cf40bE6f802a]: Small 0.17336631; Medium 1.73327195; Large 17.29370146,DOWN [0xa95BfD1e35D1Ce9906A92108A96ccEe1101aBfaa]: Small 0.24880016; Medium 2.48800158; Large 24.88001581UP [0x9C935D4d196037Fc3C37f4BB0a92cf40bE6f802a]: Small 0.17363980; Medium 1.73639798; Large 17.36397984,DOWN [0xa95BfD1e35D1Ce9906A92108A96ccEe1101aBfaa]: Small -0.00007167; Medium -0.00491953; Large -0.46180010UP [0x9C935D4d196037Fc3C37f4BB0a92cf40bE6f802a]: Small -0.00027349; Medium -0.00312604; Large -0.07027838,0x4446a16765a38a2d1a712d0d4ebf27b3efbdd50a
1,Will Elon Musk be CEO of Tesla on 1 September 2025?,https://app.seer.pm/markets/100/0x4684484388bf048b23621449a3ea4ae16faca5c6,Yes [0x27827651fb99f4d241F450e93135cAd9Fb7831F4]: Small 0.12265737; Medium 1.21786796; Large nullNo [0xbaa447Ef3fAeABEfFDB1Be2d80573459f42d45B8]: Small 0.61535097; Medium 6.05704587; Large 52.36206212,Yes [0x27827651fb99f4d241F450e93135cAd9Fb7831F4]: Small 0.12276936; Medium 1.22769358; Large 12.27693577No [0xbaa447Ef3fAeABEfFDB1Be2d80573459f42d45B8]: Small 0.62227500; Medium 6.22274996; Large 62.22749959,Yes [0x27827651fb99f4d241F450e93135cAd9Fb7831F4]: Small -0.00011199; Medium -0.00982562; Large nullNo [0xbaa447Ef3fAeABEfFDB1Be2d80573459f42d45B8]: Small -0.00692403; Medium -0.16570409; Large -9.86543747,0x4684484388bf048b23621449a3ea4ae16faca5c6
2,Will Jesus Christ return in 2025?,https://app.seer.pm/markets/100/0x696c46a21a3b3f57d651ffac126d08ad42594c24,Yes [0x3D2125D8Ad77C6F384041242EA9DDFf7ae1769De]: Small 9.99900216; Medium 99.91578221; Large 991.79402263No [0x942Ac2EBa3A87CA66dF376Bb259F49189dC7Db0a]: Small 0.10100458; Medium 1.01004569; Large 10.10044815,Yes [0x3D2125D8Ad77C6F384041242EA9DDFf7ae1769De]: Small 10.00082786; Medium 100.00827864; Large 1000.08278643No [0x942Ac2EBa3A87CA66dF376Bb259F49189dC7Db0a]: Small 0.10101468; Medium 1.01014680; Large 10.10146805,Yes [0x3D2125D8Ad77C6F384041242EA9DDFf7ae1769De]: Small -0.00182571; Medium -0.09249643; Large -8.28876380No [0x942Ac2EBa3A87CA66dF376Bb259F49189dC7Db0a]: Small -0.00001010; Medium -0.00010111; Large -0.00101990,0x696c46a21a3b3f57d651ffac126d08ad42594c24
3,How many seats will [party name] win in Japan's 2025 House of Council elections? [Seats],https://app.seer.pm/markets/100/0xbfea94c611fbe8a5353eddd94e025a2b3ad425d3,Liberal Democratic Party (LDP) [0x2b49a2Fb3c4971Ce8DF3e7F26916b078EEFf3a42]: Small 0.28748499; Medium 2.87482698; Large 28.74597283Constitutional Democratic Party of Japan (CDP) [0x1AbA86364eaCFcB77B0c94D0601a8d028655F1EB]: Small 0.50210040; Medium 5.02093384; Large 50.20232201Democratic Party For the People (DPP) [0x154fA624a3b17C9a5537Cc71A3A90E847Cde2694]: Small 0.69702494; Medium 6.97011500; Large 69.68770774Japan Innovation Party (Ishin) [0x349d9833cEFcEB2eDcF4b518800ce773d9Ac45C1]: Small 2.14479633; Medium 21.44668418; Large 214.33900584Komeito (Komei) [0x50f0A41C791D310882eBF14Ea3e0ACffbBd9f116]: Small 1.00755877; Medium 10.07530553; Large 100.72483853 Reiwa Shinsengumi (Reiwa) [0xD12a543990731FfC2BD1189ceaCbfCDE9407D628]: Small 2.48561382; Medium 24.85443344; Large 248.37397090Japanese Communist Party (JCPY) [0x1A61AbcD3E8EcA0bBD95c0c5867eCBBBC04787cf]: Small 3.74539404; Medium 37.45004462; Large 374.11128919Sanseitō (DIY) [0xC049A867eFf08764e1422355DD29F2763d1e3A12]: Small 4.55633523; Medium null; Large nullConservative Party of Japan (CPJ) [0xCB43c510647aE4200060C95d58dda4B77486Fac3]: Small 9.58475215; Medium 95.82194784; Large 955.66954503Social Democratic Party (SDP) [0x892EEd3D7DCa9AC61c02671485B04024e2E89de9]: Small 39.08463274; Medium 390.42104307; Large 3862.18534529Others [0x940d

In [12]:
sizes = [("Small", "SMALL"), ("Medium", "MEDIUM"), ("Large", "LARGE")]

stats = {key: {"cow_sum": 0.0,
               "pool_sum": 0.0,
               "diff_sum": 0.0,
               "pair_cnt": 0} 
         for _, key in sizes}

for r in results:
    for _, key in sizes:
        cow_raw  = r["cow"  ].get(f"BetAmount.{key}_sdai_price")
        pool_raw = r["pools"].get(f"BetAmount.{key}_sdai_price")

        if cow_raw is None or pool_raw is None:
            continue

        cow  = float(cow_raw)
        pool = float(pool_raw)
        diff = cow - pool

        s = stats[key]
        s["cow_sum"]  += cow
        s["pool_sum"] += pool
        s["diff_sum"] += diff
        s["pair_cnt"] += 1

summary = []
for human, key in sizes:
    s = stats[key]
    if s["pair_cnt"] == 0:
        summary.append({
            "Amount (sDAI)" : human,
            "Avg COW"       : "n/a",
            "Avg Pool"      : "n/a",
            "Avg Diff"      : "n/a",
            "Better"        : "Insufficient data",
        })
        continue

    cow_avg  = s["cow_sum"]  / s["pair_cnt"]
    pool_avg = s["pool_sum"] / s["pair_cnt"]
    diff_avg = s["diff_sum"] / s["pair_cnt"]

    if diff_avg > 0:
        better = "COW"
    elif diff_avg < 0:
        better = "Pools"
    else:
        better = "Equal"

    summary.append({
        "Amount (sDAI)" : human,
        "Avg COW"       : f"{cow_avg :.8f}",
        "Avg Pool"      : f"{pool_avg:.8f}",
        "Avg Diff"      : f"{diff_avg:+.8f}",
        "Better"        : better,
    })

df_summary = pd.DataFrame(summary)

display(df_summary)

,Amount (sDAI),Avg COW,Avg Pool,Avg Diff,Better
0,Small,2.90305292,2.92119477,-0.01814186,Pools
1,Medium,28.28884409,28.49970125,-0.21085716,Pools
2,Large,286.36560576,294.73701526,-8.37140950,Pools


In [ ]:
def parse_outcome_data(cow_text, pool_text, diff_text):
    """Parse the messy formatted text to extract structured data"""
    rows = []
    
    cow_parts = cow_text.split('||') if cow_text else []
    pool_parts = pool_text.split('||') if pool_text else []
    diff_parts = diff_text.split('||') if diff_text else []
    
    for i, cow_part in enumerate(cow_parts):
        pool_part = pool_parts[i] if i < len(pool_parts) else ""
        diff_part = diff_parts[i] if i < len(diff_parts) else ""
        
        outcome_match = re.search(r'(.*?)\s*\[(0x[a-fA-F0-9]+)\]:', cow_part.strip())
        if not outcome_match:
            continue
            
        outcome_name = outcome_match.group(1).strip()
        wrapped_address = outcome_match.group(2)
        
        def extract_size_values(text, size_name):
            pattern = f"{size_name}[^0-9]*?([0-9.]+|null)"
            match = re.search(pattern, text)
            if match:
                val = match.group(1)
                return float(val) if val != 'null' else None
            return None
        
        small_cow = extract_size_values(cow_part, "Small")
        medium_cow = extract_size_values(cow_part, "Medium") 
        large_cow = extract_size_values(cow_part, "Large")
        
        small_pool = extract_size_values(pool_part, "Small")
        medium_pool = extract_size_values(pool_part, "Medium")
        large_pool = extract_size_values(pool_part, "Large")
        
        small_diff = extract_size_values(diff_part, "Small")
        medium_diff = extract_size_values(diff_part, "Medium")
        large_diff = extract_size_values(diff_part, "Large")
        
        rows.append({
            'Outcome': outcome_name,
            'Wrapped_Address': wrapped_address,
            'Small_COW': small_cow,
            'Small_Pool': small_pool,
            'Small_Diff': small_diff,
            'Medium_COW': medium_cow,
            'Medium_Pool': medium_pool,
            'Medium_Diff': medium_diff,
            'Large_COW': large_cow,
            'Large_Pool': large_pool,
            'Large_Diff': large_diff
        })
    
    return rows

df = pd.read_csv('market_analysis.csv')

all_rows = []
for _, row in df.iterrows():
    outcome_rows = parse_outcome_data(
        row['COW Values'], 
        row['Pool Values'], 
        row['Diff Values']
    )
    
    for outcome_row in outcome_rows:
        full_row = {
            'Market_Question': row['Market Question'],
            'Market_URL': row['Market URL'],
            'Market_ID': row['Market ID'],
            **outcome_row
        }
        all_rows.append(full_row)

new_df = pd.DataFrame(all_rows)
new_df.to_csv('market_analysis_flat.csv', index=False)
print(f"Created flat CSV with {len(new_df)} rows")

Created flat CSV with 36 rows


In [ ]:
try:
    df = pd.read_csv('market_analysis_flat.csv')
except FileNotFoundError:
    import re
    
    def parse_outcome_data(cow_text, pool_text, diff_text):
        rows = []
        cow_parts = cow_text.split('||') if cow_text else []
        pool_parts = pool_text.split('||') if pool_text else []
        diff_parts = diff_text.split('||') if diff_text else []
        
        for i, cow_part in enumerate(cow_parts):
            pool_part = pool_parts[i] if i < len(pool_parts) else ""
            diff_part = diff_parts[i] if i < len(diff_parts) else ""
            
            outcome_match = re.search(r'(.*?)\s*\[(0x[a-fA-F0-9]+)\]:', cow_part.strip())
            if not outcome_match:
                continue
                
            outcome_name = outcome_match.group(1).strip()
            wrapped_address = outcome_match.group(2)
            
            def extract_size_values(text, size_name):
                pattern = f"{size_name}[^0-9]*?([0-9.]+|null)"
                match = re.search(pattern, text)
                if match:
                    val = match.group(1)
                    return float(val) if val != 'null' else None
                return None
            
            small_cow = extract_size_values(cow_part, "Small")
            medium_cow = extract_size_values(cow_part, "Medium") 
            large_cow = extract_size_values(cow_part, "Large")
            
            small_pool = extract_size_values(pool_part, "Small")
            medium_pool = extract_size_values(pool_part, "Medium")
            large_pool = extract_size_values(pool_part, "Large")
            
            small_diff = extract_size_values(diff_part, "Small")
            medium_diff = extract_size_values(diff_part, "Medium")
            large_diff = extract_size_values(diff_part, "Large")
            
            rows.append({
                'Outcome': outcome_name,
                'Wrapped_Address': wrapped_address,
                'Small_COW': small_cow,
                'Small_Pool': small_pool,
                'Small_Diff': small_diff,
                'Medium_COW': medium_cow,
                'Medium_Pool': medium_pool,
                'Medium_Diff': medium_diff,
                'Large_COW': large_cow,
                'Large_Pool': large_pool,
                'Large_Diff': large_diff
            })
        
        return rows

    orig_df = pd.read_csv('market_analysis.csv')
    all_rows = []
    for _, row in orig_df.iterrows():
        outcome_rows = parse_outcome_data(
            row['COW Values'], 
            row['Pool Values'], 
            row['Diff Values']
        )
        
        for outcome_row in outcome_rows:
            full_row = {
                'Market_Question': row['Market Question'],
                'Market_URL': row['Market URL'],
                'Market_ID': row['Market ID'],
                **outcome_row
            }
            all_rows.append(full_row)

    df = pd.DataFrame(all_rows)
    df.to_csv('market_analysis_flat.csv', index=False)

sizes = ["Small", "Medium", "Large"]
results = []

for size in sizes:
    cow_col = f"{size}_COW"
    pool_col = f"{size}_Pool"
    diff_col = f"{size}_Diff"
    
    valid_mask = df[cow_col].notna() & df[pool_col].notna()
    valid_df = df[valid_mask].copy()
    
    if len(valid_df) == 0:
        results.append({
            "Size": size,
            "Count_All": 0,
            "Count_After_Outlier_Removal": 0,
            "COW_Wins": 0,
            "Pool_Wins": 0,
            "Equal": 0,
            "COW_Win_Rate": "n/a",
            "Pool_Win_Rate": "n/a",
            "Avg_COW": "n/a",
            "Avg_Pool": "n/a", 
            "Avg_Diff": "n/a",
            "Avg_Pct_Diff": "n/a",
            "Avg_Pct_When_COW_Wins": "n/a",
            "Avg_Pct_When_Pool_Wins": "n/a",
            "Better": "Insufficient data"
        })
        continue
    
    valid_df[f'{size}_Calculated_Diff'] = valid_df[cow_col] - valid_df[pool_col]
    valid_df[f'{size}_Pct_Diff'] = (valid_df[cow_col] - valid_df[pool_col]) / valid_df[pool_col] * 100
    
    cow_wins_all = (valid_df[f'{size}_Calculated_Diff'] > 0).sum()
    pool_wins_all = (valid_df[f'{size}_Calculated_Diff'] < 0).sum()
    equal_all = (valid_df[f'{size}_Calculated_Diff'] == 0).sum()
    
    cow_win_mask = valid_df[f'{size}_Calculated_Diff'] > 0
    pool_win_mask = valid_df[f'{size}_Calculated_Diff'] < 0
    
    avg_pct_cow_wins = valid_df[cow_win_mask][f'{size}_Pct_Diff'].mean() if cow_wins_all > 0 else 0
    avg_pct_pool_wins = valid_df[pool_win_mask][f'{size}_Pct_Diff'].mean() if pool_wins_all > 0 else 0
    
    diff_values = valid_df[f'{size}_Calculated_Diff']
    q10 = diff_values.quantile(0.1)
    q90 = diff_values.quantile(0.9)
    
    filtered_df = valid_df[(diff_values >= q10) & (diff_values <= q90)]
    
    if len(filtered_df) == 0:
        results.append({
            "Size": size,
            "Count_All": len(valid_df),
            "Count_After_Outlier_Removal": 0,
            "COW_Wins": cow_wins_all,
            "Pool_Wins": pool_wins_all,
            "Equal": equal_all,
            "COW_Win_Rate": f"{cow_wins_all/len(valid_df)*100:.1f}%" if len(valid_df) > 0 else "n/a",
            "Pool_Win_Rate": f"{pool_wins_all/len(valid_df)*100:.1f}%" if len(valid_df) > 0 else "n/a",
            "Avg_COW": "n/a",
            "Avg_Pool": "n/a",
            "Avg_Diff": "n/a",
            "Avg_Pct_Diff": "n/a",
            "Avg_Pct_When_COW_Wins": f"{avg_pct_cow_wins:+.2f}%" if cow_wins_all > 0 else "n/a",
            "Avg_Pct_When_Pool_Wins": f"{avg_pct_pool_wins:+.2f}%" if pool_wins_all > 0 else "n/a",
            "Better": "Insufficient data"
        })
        continue
    
    avg_cow = filtered_df[cow_col].mean()
    avg_pool = filtered_df[pool_col].mean()
    avg_diff = filtered_df[f'{size}_Calculated_Diff'].mean()
    avg_pct_diff = filtered_df[f'{size}_Pct_Diff'].mean()
    
    cow_wins_filtered = (filtered_df[f'{size}_Calculated_Diff'] > 0).sum()
    pool_wins_filtered = (filtered_df[f'{size}_Calculated_Diff'] < 0).sum()
    equal_filtered = (filtered_df[f'{size}_Calculated_Diff'] == 0).sum()
    
    if avg_diff > 0:
        better = "COW"
    elif avg_diff < 0:
        better = "Pools"
    else:
        better = "Equal"
    
    results.append({
        "Size": size,
        "Count_All": len(valid_df),
        "Count_After_Outlier_Removal": len(filtered_df),
        "COW_Wins": cow_wins_all,
        "Pool_Wins": pool_wins_all,
        "Equal": equal_all,
        "COW_Win_Rate": f"{cow_wins_all/len(valid_df)*100:.1f}%",
        "Pool_Win_Rate": f"{pool_wins_all/len(valid_df)*100:.1f}%",
        "COW_Wins_Filtered": cow_wins_filtered,
        "Pool_Wins_Filtered": pool_wins_filtered,
        "COW_Win_Rate_Filtered": f"{cow_wins_filtered/len(filtered_df)*100:.1f}%",
        "Pool_Win_Rate_Filtered": f"{pool_wins_filtered/len(filtered_df)*100:.1f}%",
        "Avg_COW": f"{avg_cow:.8f}",
        "Avg_Pool": f"{avg_pool:.8f}",
        "Avg_Diff": f"{avg_diff:+.8f}",
        "Avg_Pct_Diff": f"{avg_pct_diff:+.2f}%",
        "Avg_Pct_When_COW_Wins": f"{avg_pct_cow_wins:+.2f}%" if cow_wins_all > 0 else "n/a",
        "Avg_Pct_When_Pool_Wins": f"{avg_pct_pool_wins:+.2f}%" if pool_wins_all > 0 else "n/a",
        "Better": better
    })

summary_df = pd.DataFrame(results)
print("Enhanced Summary with win-specific percentage averages:")
print(summary_df.to_string(index=False))

compact_results = []
for result in results:
    compact_results.append({
        "Size": result["Size"],
        "Total_Count": result["Count_All"],
        "COW_Wins": f"{result['COW_Wins']} ({result['COW_Win_Rate']})",
        "Pool_Wins": f"{result['Pool_Wins']} ({result['Pool_Win_Rate']})",
        "Avg_%_When_COW_Wins": result["Avg_Pct_When_COW_Wins"],
        "Avg_%_When_Pool_Wins": result["Avg_Pct_When_Pool_Wins"],
        "Overall_Avg_%_Diff": result["Avg_Pct_Diff"],
        "Better": result["Better"]
    })

compact_df = pd.DataFrame(compact_results)
print("\n\nCompact Summary with Win-Specific Percentage Averages:")
print(compact_df.to_string(index=False))

Enhanced Summary with win-specific percentage averages:
  Size  Count_All  Count_After_Outlier_Removal  COW_Wins  Pool_Wins  Equal COW_Win_Rate Pool_Win_Rate  COW_Wins_Filtered  Pool_Wins_Filtered COW_Win_Rate_Filtered Pool_Win_Rate_Filtered      Avg_COW     Avg_Pool    Avg_Diff Avg_Pct_Diff Avg_Pct_When_COW_Wins Avg_Pct_When_Pool_Wins Better
 Small         30                           24         0         30      0         0.0%        100.0%                  0                  24                  0.0%                 100.0%   3.34486473   3.34789067 -0.00302594       -0.60%                   n/a                 -1.10%  Pools
Medium         29                           23         0         29      0         0.0%        100.0%                  0                  23                  0.0%                 100.0%  34.72867593  34.83415574 -0.10547981       -2.38%                   n/a                 -7.24%  Pools
 Large         28                           22         0         28      0   

In [ ]:
try:
    df = pd.read_csv('market_analysis_flat.csv')
except FileNotFoundError:
    import re
    
    def parse_outcome_data(cow_text, pool_text, diff_text):
        rows = []
        cow_parts = cow_text.split('||') if cow_text else []
        pool_parts = pool_text.split('||') if pool_text else []
        diff_parts = diff_text.split('||') if diff_text else []
        
        for i, cow_part in enumerate(cow_parts):
            pool_part = pool_parts[i] if i < len(pool_parts) else ""
            diff_part = diff_parts[i] if i < len(diff_parts) else ""
            
            outcome_match = re.search(r'(.*?)\s*\[(0x[a-fA-F0-9]+)\]:', cow_part.strip())
            if not outcome_match:
                continue
                
            outcome_name = outcome_match.group(1).strip()
            wrapped_address = outcome_match.group(2)
            
            def extract_size_values(text, size_name):
                pattern = f"{size_name}[^0-9]*?([0-9.]+|null)"
                match = re.search(pattern, text)
                if match:
                    val = match.group(1)
                    return float(val) if val != 'null' else None
                return None
            
            small_cow = extract_size_values(cow_part, "Small")
            medium_cow = extract_size_values(cow_part, "Medium") 
            large_cow = extract_size_values(cow_part, "Large")
            
            small_pool = extract_size_values(pool_part, "Small")
            medium_pool = extract_size_values(pool_part, "Medium")
            large_pool = extract_size_values(pool_part, "Large")
            
            small_diff = extract_size_values(diff_part, "Small")
            medium_diff = extract_size_values(diff_part, "Medium")
            large_diff = extract_size_values(diff_part, "Large")
            
            rows.append({
                'Outcome': outcome_name,
                'Wrapped_Address': wrapped_address,
                'Small_COW': small_cow,
                'Small_Pool': small_pool,
                'Small_Diff': small_diff,
                'Medium_COW': medium_cow,
                'Medium_Pool': medium_pool,
                'Medium_Diff': medium_diff,
                'Large_COW': large_cow,
                'Large_Pool': large_pool,
                'Large_Diff': large_diff
            })
        
        return rows

    # Parse original CSV
    orig_df = pd.read_csv('market_analysis.csv')
    all_rows = []
    for _, row in orig_df.iterrows():
        outcome_rows = parse_outcome_data(
            row['COW Values'], 
            row['Pool Values'], 
            row['Diff Values']
        )
        
        for outcome_row in outcome_rows:
            full_row = {
                'Market_Question': row['Market Question'],
                'Market_URL': row['Market URL'],
                'Market_ID': row['Market ID'],
                **outcome_row
            }
            all_rows.append(full_row)

    df = pd.DataFrame(all_rows)
    df.to_csv('market_analysis_flat.csv', index=False)

# Add calculated differences for all sizes
df['Small_Calc_Diff'] = df['Small_COW'] - df['Small_Pool']
df['Medium_Calc_Diff'] = df['Medium_COW'] - df['Medium_Pool']
df['Large_Calc_Diff'] = df['Large_COW'] - df['Large_Pool']

df['Small_Pct_Diff'] = (df['Small_COW'] - df['Small_Pool']) / df['Small_Pool'] * 100
df['Medium_Pct_Diff'] = (df['Medium_COW'] - df['Medium_Pool']) / df['Medium_Pool'] * 100
df['Large_Pct_Diff'] = (df['Large_COW'] - df['Large_Pool']) / df['Large_Pool'] * 100

print("=== ANALYSIS: Why COW wins so much ===\n")

# Let's look at the distribution of differences for each size
for size in ["Small", "Medium", "Large"]:
    print(f"\n--- {size} Size Analysis ---")
    
    cow_col = f"{size}_COW"
    pool_col = f"{size}_Pool"
    diff_col = f"{size}_Calc_Diff"
    pct_diff_col = f"{size}_Pct_Diff"
    
    # Get valid rows
    valid_mask = df[cow_col].notna() & df[pool_col].notna()
    valid_df = df[valid_mask].copy()
    
    if len(valid_df) == 0:
        print(f"No valid data for {size}")
        continue
    
    cow_wins = (valid_df[diff_col] > 0).sum()
    pool_wins = (valid_df[diff_col] < 0).sum()
    equal = (valid_df[diff_col] == 0).sum()
    
    print(f"Total valid comparisons: {len(valid_df)}")
    print(f"COW wins: {cow_wins} ({cow_wins/len(valid_df)*100:.1f}%)")
    print(f"Pool wins: {pool_wins} ({pool_wins/len(valid_df)*100:.1f}%)")
    print(f"Equal: {equal} ({equal/len(valid_df)*100:.1f}%)")
    
    cow_win_examples = valid_df[valid_df[diff_col] > 0].nlargest(5, pct_diff_col)
    print(f"\nTop 5 COW wins by percentage:")
    for idx, row in cow_win_examples.iterrows():
        print(f"  {row['Outcome'][:30]:<30} | COW: {row[cow_col]:.4f} | Pool: {row[pool_col]:.4f} | Diff: {row[pct_diff_col]:+.1f}%")
    
    pool_win_examples = valid_df[valid_df[diff_col] < 0].nsmallest(5, pct_diff_col)
    print(f"\nTop 5 Pool wins by percentage:")
    for idx, row in pool_win_examples.iterrows():
        print(f"  {row['Outcome'][:30]:<30} | COW: {row[cow_col]:.4f} | Pool: {row[pool_col]:.4f} | Diff: {row[pct_diff_col]:+.1f}%")
    
    print(f"\nPatterns in COW wins:")
    cow_wins_df = valid_df[valid_df[diff_col] > 0]
    
    outcome_counts = cow_wins_df['Outcome'].value_counts().head(10)
    print("Most common outcomes where COW wins:")
    for outcome, count in outcome_counts.items():
        total_for_outcome = valid_df[valid_df['Outcome'] == outcome].shape[0]
        print(f"  {outcome[:40]:<40}: {count}/{total_for_outcome} ({count/total_for_outcome*100:.1f}%)")
    
    print(f"\nCOW win magnitude distribution:")
    cow_win_pcts = cow_wins_df[pct_diff_col]
    print(f"  Median COW advantage: {cow_win_pcts.median():.2f}%")
    print(f"  Mean COW advantage: {cow_win_pcts.mean():.2f}%")
    print(f"  75th percentile: {cow_win_pcts.quantile(0.75):.2f}%")
    print(f"  95th percentile: {cow_win_pcts.quantile(0.95):.2f}%")
    
    print(f"\nPool win magnitude distribution:")
    pool_win_pcts = valid_df[valid_df[diff_col] < 0][pct_diff_col]
    if len(pool_win_pcts) > 0:
        print(f"  Median Pool advantage: {pool_win_pcts.median():.2f}%")
        print(f"  Mean Pool advantage: {pool_win_pcts.mean():.2f}%")
        print(f"  25th percentile: {pool_win_pcts.quantile(0.25):.2f}%")
        print(f"  5th percentile: {pool_win_pcts.quantile(0.05):.2f}%")
    else:
        print("  No pool wins to analyze")

print("\n=== SYSTEMATIC PATTERN ANALYSIS ===")

print("\nMarkets where COW consistently wins:")
market_analysis = []
for market_id in df['Market_ID'].unique():
    market_df = df[df['Market_ID'] == market_id]
    
    total_comparisons = 0
    cow_wins = 0
    
    for size in ["Small", "Medium", "Large"]:
        valid_mask = market_df[f"{size}_COW"].notna() & market_df[f"{size}_Pool"].notna()
        valid_market_df = market_df[valid_mask]
        
        if len(valid_market_df) > 0:
            size_cow_wins = (valid_market_df[f"{size}_Calc_Diff"] > 0).sum()
            total_comparisons += len(valid_market_df)
            cow_wins += size_cow_wins
    
    if total_comparisons > 0:
        cow_win_rate = cow_wins / total_comparisons
        market_analysis.append({
            'Market_ID': market_id,
            'Market_Question': market_df['Market_Question'].iloc[0][:60],
            'Total_Comparisons': total_comparisons,
            'COW_Wins': cow_wins,
            'COW_Win_Rate': cow_win_rate
        })

market_analysis_df = pd.DataFrame(market_analysis)
market_analysis_df = market_analysis_df.sort_values('COW_Win_Rate', ascending=False)

print("Markets sorted by COW win rate:")
for idx, row in market_analysis_df.head(10).iterrows():
    print(f"  {row['COW_Win_Rate']*100:5.1f}% | {row['COW_Wins']:2d}/{row['Total_Comparisons']:2d} | {row['Market_Question']}")

=== ANALYSIS: Why COW wins so much ===


--- Small Size Analysis ---
Total valid comparisons: 30
COW wins: 0 (0.0%)
Pool wins: 30 (100.0%)
Equal: 0 (0.0%)

Top 5 COW wins by percentage:

Top 5 Pool wins by percentage:
  Sanseitō (DIY)                 | COW: 4.5563 | Pool: 4.9867 | Diff: -8.6%
  UP                             | COW: 0.1825 | Pool: 0.1996 | Diff: -8.6%
  DOWN                           | COW: 0.1274 | Pool: 0.1366 | Diff: -6.7%
  DOWN                           | COW: 0.1447 | Pool: 0.1471 | Diff: -1.7%
  Others                         | COW: 1.4991 | Pool: 1.5232 | Diff: -1.6%

Patterns in COW wins:
Most common outcomes where COW wins:

COW win magnitude distribution:
  Median COW advantage: nan%
  Mean COW advantage: nan%
  75th percentile: nan%
  95th percentile: nan%

Pool win magnitude distribution:
  Median Pool advantage: -0.03%
  Mean Pool advantage: -1.10%
  25th percentile: -1.13%
  5th percentile: -7.72%

--- Medium Size Analysis ---
Total valid comparisons: 29
